<a href="https://colab.research.google.com/github/andresbadillo/aenc_tfroc/blob/master/aenc_tfroc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Consumo = aenc / Factor de pérdidas del `-día-`

En este pieza se código se cargan los archivos aenc y tfroc del mismo día seleccionado y se realiza la división de cada valor horario por el factor de perdidas

In [ ]:
import pandas as pd

# Cargar los archivos desde Google Colab
from google.colab import files

# Subir los archivos
print("Por favor, sube el archivo aenc")
aenc_file = files.upload()

print("\nPor favor, sube el archivo tfroc")
tfroc_file = files.upload()

# Leer los archivos con el separador ";"
aenc_file_path = list(aenc_file.keys())[0]
tfroc_file_path = list(tfroc_file.keys())[0]

aenc_df = pd.read_csv(aenc_file_path, sep=";", encoding='latin1')
tfroc_df = pd.read_csv(tfroc_file_path, sep=";", encoding='latin1')

# Renombrar columnas para facilitar la unión
aenc_df.rename(columns={"CODIGO SIC": "CODIGO FRONTERA"}, inplace=True)

# Unir los dos dataframes utilizando "CODIGO FRONTERA"
merged_df = pd.merge(aenc_df, tfroc_df[["CODIGO FRONTERA", "FACTOR DE PERDIDAS"]], on="CODIGO FRONTERA", how="inner")

# Dividir los valores de aenc horario por el FACTOR DE PERDIDAS
horas = [col for col in aenc_df.columns if "HORA" in col]
for hora in horas:
    merged_df[hora] = merged_df[hora] / merged_df["FACTOR DE PERDIDAS"]

# Agregar columnas adicionales al archivo de salida
merged_df.rename(columns={"CODIGO PROPIO": "NOMBRE FRONTERA"}, inplace=True)
output_df = merged_df[["CODIGO FRONTERA", "NOMBRE FRONTERA", "TIPO DE AGRUPACIÓN", "IMPO - EXPO"] + horas]

# Agregar columna de total de consumo por frontera
output_df["TOTAL CONSUMO"] = output_df[horas].sum(axis=1)

# Exportar a un archivo CSV
output_file_name = "consumos_1201.csv"
output_df.to_csv(output_file_name, index=False, sep=";")

# Descargar el archivo resultante
files.download(output_file_name)
print(f"\nEl archivo {output_file_name} ha sido descargado")

## Consumo = aenc / Factor de pérdidas del `-Mes-`

En este pieza se código se cargan los archivos aenc y tfroc del mismo mes y se realiza la división de cada valor horario por el factor de perdidas. Hay dos archivos exportados:
- Archivo que consolida todos los aenc del mes: aenc_consolidado_`{mm}`_`{aaaa}`.csv
- Archivo que consolida los consumos del mes: consumos_`{mm}`_`{aaaa}`.csv

In [ ]:
!pip install holidays_co

import pandas as pd
import os
from datetime import datetime
import holidays_co

# Configurar el año como variable
anio_actual = 2025  # Cambiar este valor según sea necesario

# Cargar los archivos desde Google Colab
from google.colab import files

# Subir los archivos
aenc_files = []
tfroc_files = []

print("Por favor, sube todos los archivos de 'aenc' del mes")
aenc_uploaded = files.upload()
aenc_files.extend(aenc_uploaded.keys())

print("\nPor favor, sube todos los archivos de 'tfroc' del mes")
tfroc_uploaded = files.upload()
tfroc_files.extend(tfroc_uploaded.keys())

# Inicializar dataframes vacíos para consolidar los resultados
consolidated_aenc_df = pd.DataFrame()
consolidated_df = pd.DataFrame()

# Obtener los festivos de Colombia para el año configurado
festivos = holidays_co.get_colombia_holidays_by_year(anio_actual)
#print(f"Festivos del año {anio_actual}: {festivos}")

# Función para obtener el nombre del día en español
def obtener_dia_semana(fecha):
    dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
    dia_semana = datetime.strptime(fecha, "%d-%m-%Y").weekday()
    return dias_semana[dia_semana]

# Función para determinar si un día es festivo o no, considerando domingos como festivos
def es_festivo(fecha):
    fecha_obj = datetime.strptime(fecha, "%d-%m-%Y").date()
    if holidays_co.is_holiday_date(fecha_obj) or fecha_obj.weekday() == 6:  # 6 representa domingo
        return "Festivo"
    return "Ordinario"

# Diccionario de mapeo para la columna "OR"
mapeo_or = {
    "CUNM": "ENEL",
    "SOLM": "AIRE",
    "SANM": "ESSA",
    "MARM": "AFINIA",
    "NSAM": "ESSA",
    "BOYM": "EBSA",
    "CASM": "ENERCA",
    "METM": "EMSA",
    "QUIM": "EDEQ",
    "RUIM": "RUITOQUE",
    "CHOM": "DISPAC",
    "CLOM": "EMCALI"
}

# Diccionario de mapeo para la columna "NT"
mapeo_nt = {
    11.4: 2,
    13.8: 2,
    0.2: 1,
    34.5: 3,
    13.2: 2,
    0.1: 1,
    0.22: 1,
    12: 2,
    0.11: 1,
    0.12: 1,
    0.5: 1,
    2: 2,
    1: 1,
    0.44: 1
}

# Procesar archivos día a día
for aenc_file_path in aenc_files:
    # Extraer el día y el mes del nombre del archivo
    day = os.path.splitext(aenc_file_path)[0][-4:]  # Los últimos 4 caracteres son el día
    date = f"{day[2:]}-{day[:2]}-{anio_actual}"  # Formatear como dd-mm-aaaa

    # Buscar el archivo tfroc correspondiente
    tfroc_file_path = next((f for f in tfroc_files if day in f), None)
    if tfroc_file_path is None:
        print(f"No se encontró un archivo tfroc correspondiente para el día {day}.")
        continue

    # Leer los archivos con el separador ";"
    aenc_df = pd.read_csv(aenc_file_path, sep=";", encoding='latin1')
    tfroc_df = pd.read_csv(tfroc_file_path, sep=";", encoding='latin1')

    # Renombrar columnas para facilitar la unión
    aenc_df.rename(columns={"CODIGO SIC": "CODIGO FRONTERA"}, inplace=True)

    # Agregar columnas adicionales al dataframe de aenc
    horas = [col for col in aenc_df.columns if "HORA" in col]
    aenc_df.insert(0, "FECHA", date)
    aenc_df.insert(1, "DIA", obtener_dia_semana(date))
    aenc_df.insert(2, "TIPO_DIA", es_festivo(date))
    aenc_df["TOTAL CONSUMO"] = aenc_df.loc[:, horas].sum(axis=1)

    # Consolidar los datos de aenc
    consolidated_aenc_df = pd.concat([consolidated_aenc_df, aenc_df], ignore_index=True)

    # Unir los dos dataframes utilizando "CODIGO FRONTERA"
    merged_df = pd.merge(aenc_df, tfroc_df[["CODIGO FRONTERA", "FACTOR DE PERDIDAS", "MERCADO COMERCIALIZACIÓN QUE EXPORTA", "NIVEL DE TENSION"]], on="CODIGO FRONTERA", how="inner")

    # Dividir los valores de consumo horario por el FACTOR DE PERDIDAS
    for hora in horas:
        merged_df[hora] = merged_df[hora] / merged_df["FACTOR DE PERDIDAS"]

    # Agregar columnas adicionales al archivo de salida
    merged_df.rename(columns={"CODIGO PROPIO": "NOMBRE FRONTERA"}, inplace=True)
    daily_df = merged_df.loc[:, ["CODIGO FRONTERA", "NOMBRE FRONTERA", "MERCADO COMERCIALIZACIÓN QUE EXPORTA", "NIVEL DE TENSION","TIPO DE AGRUPACIÓN", "IMPO - EXPO"] + horas]

    # Agregar columna de fecha, día y tipo de día
    daily_df.insert(0, "FECHA", date)
    daily_df.insert(1, "DIA", obtener_dia_semana(date))
    daily_df.insert(2, "TIPO_DIA", es_festivo(date))

    # Agregar columna "OR" basada en el mapeo
    daily_df.insert(6, "OR", daily_df["MERCADO COMERCIALIZACIÓN QUE EXPORTA"].map(mapeo_or))

    # Agregar columna "NT" basada en el mapeo
    daily_df.insert(8, "NT", daily_df["NIVEL DE TENSION"].map(mapeo_nt))

    # Agregar columna de total de consumo por frontera
    daily_df["TOTAL CONSUMO"] = daily_df.loc[:, horas].sum(axis=1)

    # Agregar los resultados al dataframe consolidado
    consolidated_df = pd.concat([consolidated_df, daily_df], ignore_index=True)

# Exportar el archivo de aenc consolidado
mes_archivos = aenc_files[0][4:6]  # Obtener el mes de los archivos cargados
output_aenc_file_name = f"aenc_consolidado_{mes_archivos}_{anio_actual}.csv"
consolidated_aenc_df.to_csv(output_aenc_file_name, index=False, sep=";", encoding='utf-8-sig')
files.download(output_aenc_file_name)
print(f"\nEl archivo {output_aenc_file_name} ha sido descargado")

# Exportar el archivo de consumos consolidado
output_file_name = f"consumos_{mes_archivos}_{anio_actual}.csv"
consolidated_df.to_csv(output_file_name, index=False, sep=";", encoding='utf-8-sig')
files.download(output_file_name)
print(f"\nEl archivo {output_file_name} ha sido descargado")

Por favor, sube todos los archivos de 'aenc' del mes


Saving aenc0101.Tx2 to aenc0101.Tx2
Saving aenc0102.Tx2 to aenc0102.Tx2
Saving aenc0103.Tx2 to aenc0103.Tx2
Saving aenc0104.Tx2 to aenc0104.Tx2
Saving aenc0105.Tx2 to aenc0105.Tx2
Saving aenc0106.Tx2 to aenc0106.Tx2
Saving aenc0107.Tx2 to aenc0107.Tx2
Saving aenc0108.Tx2 to aenc0108.Tx2
Saving aenc0109.Tx2 to aenc0109.Tx2
Saving aenc0110.Tx2 to aenc0110.Tx2
Saving aenc0111.Tx2 to aenc0111.Tx2
Saving aenc0112.Tx2 to aenc0112.Tx2
Saving aenc0113.Tx2 to aenc0113.Tx2
Saving aenc0114.Tx2 to aenc0114.Tx2
Saving aenc0115.Tx2 to aenc0115.Tx2
Saving aenc0116.Tx2 to aenc0116.Tx2
Saving aenc0117.Tx2 to aenc0117.Tx2
Saving aenc0118.Tx2 to aenc0118.Tx2
Saving aenc0119.Tx2 to aenc0119.Tx2
Saving aenc0120.Tx2 to aenc0120.Tx2
Saving aenc0121.Tx2 to aenc0121.Tx2
Saving aenc0122.Tx2 to aenc0122.Tx2

Por favor, sube todos los archivos de 'tfroc' del mes


Saving tfroc0101.Tx2 to tfroc0101.Tx2
Saving tfroc0102.Tx2 to tfroc0102.Tx2
Saving tfroc0103.Tx2 to tfroc0103.Tx2
Saving tfroc0104.Tx2 to tfroc0104.Tx2
Saving tfroc0105.Tx2 to tfroc0105.Tx2
Saving tfroc0106.Tx2 to tfroc0106.Tx2
Saving tfroc0107.Tx2 to tfroc0107.Tx2
Saving tfroc0108.Tx2 to tfroc0108.Tx2
Saving tfroc0109.Tx2 to tfroc0109.Tx2
Saving tfroc0110.Tx2 to tfroc0110.Tx2
Saving tfroc0111.Tx2 to tfroc0111.Tx2
Saving tfroc0112.Tx2 to tfroc0112.Tx2
Saving tfroc0113.Tx2 to tfroc0113.Tx2
Saving tfroc0114.Tx2 to tfroc0114.Tx2
Saving tfroc0115.Tx2 to tfroc0115.Tx2
Saving tfroc0116.Tx2 to tfroc0116.Tx2
Saving tfroc0117.Tx2 to tfroc0117.Tx2
Saving tfroc0118.Tx2 to tfroc0118.Tx2
Saving tfroc0119.Tx2 to tfroc0119.Tx2
Saving tfroc0120.Tx2 to tfroc0120.Tx2
Saving tfroc0121.Tx2 to tfroc0121.Tx2
Saving tfroc0122.Tx2 to tfroc0122.Tx2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


El archivo aenc_consolidado_01_2025.csv ha sido descargado


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


El archivo consumos_01_2025.csv ha sido descargado
